In [ ]:
from scipy.io import loadmat
from pandas import factorize
import pickle
import numpy as np
import random
from scipy import signal
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import os,random
os.environ["KERAS_BACKEND"] = "tensorflow"
import math
import numpy as np
import matplotlib.pyplot as plt
import pickle, random, sys, keras

import keras.models as models

from keras.layers import Input,Conv2D, Dense, MaxPool2D, Flatten,ZeroPadding1D,MaxPool1D
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D,ZeroPadding2D
from keras.layers import Reshape, Dense, Dropout, Activation

from keras.layers import  ZeroPadding1D, MaxPool1D
from keras.regularizers import *
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

In [ ]:
# RadioML2016.10a/10b or MIGOU MOD

def load_dataset(dataset_location):
    """
    Load dataset and extract needed data

    Input:
        dataset_location: specify where the file is stored and its name

    Output:
        snrs: list of the SNR range in dataset [-20 to 18]
        X: array of the measured I/Q data [num_of_samples, 128, 2]
        modulations: list of the modulations in this dataset
        one_hot_encode: one_hot encoded data - the other maps the order of the mods
        lbl_SNR: list of each snr (for plotting)
    """

    snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], dataset_location.keys())))), [1,0])

    X = []; I = []; Q = []; lbl = [];

    for mod in mods:
        for snr in snrs:
            X.append(dataset_location[(mod,snr)])
            for i in range(dataset_location[(mod,snr)].shape[0]):
                lbl.append((mod,snr))
    X = np.vstack(X); lbl=np.vstack(lbl)

    X = np.transpose(X[:,:,:],(0,2,1))

    # One-hot-encoding
    Y = [];
    for i in range(len(lbl)):
        mod = (lbl[i,0])
        Y.append(mod)

    mapping = {}
    for x in range(len(mods)):
        mapping[mods[x]] = x

    ## integer representation
    for x in range(len(Y)):
        Y[x] = mapping[Y[x]]

    one_hot_encode = to_categorical(Y)

    # Assign SNR value to each vector
    repeat_n = X.shape[0]/len(mods)/len(snrs)
    repeat_n_mod = len(mods)
    lbl_SNR = np.tile(np.repeat(snrs, repeat_n), repeat_n_mod)



    return snrs, X, mods, one_hot_encode, lbl_SNR

In [ ]:
#X.shape


In [ ]:
# RML2016.10b / just for the way it is saved in my GoogleDrive

def load_RMLb_dataset(X, lbl):
    mods = np.unique(lbl[:,0])
    snrs = np.unique(lbl[:,1])
    snrs = list(map(int, snrs))
    snrs.sort()

    # One-hot encoding
    Y = [];
    for i in range(len(lbl)):
        mod = (lbl[i,0])
        Y.append(mod)

    mapping = {}
    for x in range(len(mods)):
        mapping[mods[x]] = x

    ## integer representation
    for x in range(len(Y)):
        Y[x] = mapping[Y[x]]

    one_hot_encode = to_categorical(Y)


    # Assign SNR value to each vector
    repeat_n = X.shape[0]/len(mods)/len(snrs)
    repeat_n_mod = len(mods)
    lbl_SNR = np.tile(np.repeat(snrs, repeat_n), repeat_n_mod)

    X = X

    return snrs, X, mods, one_hot_encode, lbl_SNR


In [ ]:
def train_test_valid_split(X, one_hot, train_split=0.7, valid_split=0.15, test_split=0.15):

    """
    Train-Test split the data

    Input:
        X: X data
        one_hot: Y data encoded to one_hot
        train_split (default 0.7)
        valid_split (default 0.15)
        test_split (default 0.15)
        train_split : valid_split : test_split - ratio for splitting the dataset

        NOTE: the ratio split must be a sum of 1!

    Output:
        train_idx: indexes from X assinged to train data
        valid_idx: indexes from X assinged to validation data
        test_idx: indexes from X assinged to test data
        X_train: X data assigned for training
        X_valid: X data assigned for validation
        X_test: X data assigned for testing
        Y_train: one-hot encoded Y data assigned for training
        Y_valid: one-hot encoded Y data assigned for validation
        Y_test: one-hot encoded Y data assigned for testing
    """

    # Set random seed
    np.random.seed(42)
    random.seed(42)

    # Get the number of samples
    n_examples = X.shape[0]
    n_train = int(n_examples * train_split)
    n_valid = int(n_examples * valid_split)
    n_test = int(n_examples * test_split)

    # Get indexes of train data
    train_idx = np.random.choice(range(0, n_examples), size=n_train, replace=False)

    # Left indexes for valid and test sets
    left_idx= list(set(range(0, n_examples)) - set(train_idx))

    # Get indexes for the left indexes of the X data
    val = np.random.choice(range(0, (n_valid+n_test)), size=(n_valid), replace=False)
    test = list(set(range(0, len(left_idx))) - set(val))

    # Assign indeces for validation to left indexes
    valid_idx = []
    for i in val:
        val_idx = left_idx[i]
        valid_idx.append(val_idx)

    # Get the test set as the rest indexes
    test_idx = []
    for i in test:
        tst_idx = left_idx[i]
        test_idx.append(tst_idx)

    # Shuffle the valid_idx and test_idx
    random.shuffle(valid_idx)
    random.shuffle(test_idx)

    # Assing the indexes to the X and Y data to create train and test sets
    X_train = X[train_idx]
    X_valid = X[valid_idx]
    X_test = X[test_idx]
    Y_train = one_hot[train_idx]
    Y_valid = one_hot[valid_idx]
    Y_test = one_hot[test_idx]

    return train_idx, valid_idx, test_idx, X_train, X_valid, X_test, Y_train, Y_valid, Y_test



In [ ]:
def normalize_data(X_train, X_valid, X_test):
        # mean-std normalization

    mean = X_train[:,:,:].mean(axis=0)
    X_train[:,:,:] -= mean
    std = X_train[:,:,:].std(axis=0)
    X_train[:,:,:] /= std


    X_valid[:,:,:] -= mean
    X_valid[:,:,:] /= std

    X_test[:,:,:] -= mean
    X_test[:,:,:] /= std

    return X_train, X_valid, X_test

In [ ]:
def return_indices_of_a(a, b):
    """
    Compare two lists a, b for same items and return indeces
    of the item in list a

    a:    List of items, its indeces will be returned
    b:    List of items to search for in list a

    Credit: https://stackoverflow.com/users/97248/pts ; https://stackoverflow.com/questions/10367020/compare-two-lists-in-python-and-return-indices-of-matched-values
    """
    b_set = set(b)
    return [i for i, v in enumerate(a) if v in b_set]

In [ ]:
def show_confusion_matrix(validations, predictions, matrix_snr, save=True):
    """
    Plot confusion matrix

    validations:    True Y labels
    predictions:    Predicted Y labels of your model
    matrix_snr:     SNR information for plot's titel
    """

    cm = confusion_matrix(validations, predictions)
    # Normalise
    cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(cmn, cmap='Blues', annot=True, fmt='.2f', xticklabels=mods, yticklabels=mods)
    sns.set(font_scale=1.3)
    if matrix_snr == None:
        plt.title("Confusion Matrix")
    else:
        plt.title("Confusion Matrix \n" + str(matrix_snr) + "dB")
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    if save == True:
        plt.savefig('VGGMatrix.png',dpi=300)

    plt.show(block=True)


In [ ]:
def All_SNR_show_confusion_matrix(X_test, save=True):
    """
    Plot confusion matrix of all SNRs in one

    X_test:   X_test data
    """
    prediction = model.predict(X_test)

    Y_Pred = []; Y_Test = [];

    for i in range(len(prediction[:,0])):
        Y_Pred.append(np.argmax(prediction[i,:]))
        Y_Test.append(np.argmax(Y_test[i]))

    show_confusion_matrix(Y_Pred, Y_Test, None, save)

In [ ]:
def SNR_show_confusion_matrix(in_snr, lbl_SNR, X_test, save=True):
    """
    Plot confusion matrices of chosen SNRs

    in_snr:   must be list of SNRs
    X_test:   X_test data
    """
    for snr in in_snr:
        matrix_snr = snr
        m_snr = matrix_snr;

        Y_Pred = []; Y_Test = []; Y_Pred_SNR = []; Y_Test_SNR = [];
        matrix_snr_index = [];

        prediction = model.predict(X_test)

        for i in range(len(prediction[:,0])):
            Y_Pred.append(np.argmax(prediction[i,:]))
            Y_Test.append(np.argmax(Y_test[i]))

        for i in range(len(lbl_SNR)):
            if int(lbl_SNR[i]) == m_snr:
                matrix_snr_index.append(i)

        indeces_of_Y_test = return_indices_of_a(test_idx, matrix_snr_index)

        for i in indeces_of_Y_test:
            Y_Pred_SNR.append(Y_Pred[i])
            Y_Test_SNR.append(Y_Test[i])
        show_confusion_matrix(Y_Pred_SNR, Y_Test_SNR, matrix_snr, save)

In [ ]:
def plot_split_distribution(mods, Y_train, Y_valid, Y_test):

    x = np.arange(len(mods))  # the label locations
    width = 1  # the width of the bars

    fig, ax = plt.subplots()
    bar1 = ax.bar(x-width*0.3, np.count_nonzero(Y_train == 1, axis=0), width*0.3, label = "Train" )
    bar2 = ax.bar(x , np.count_nonzero(Y_valid == 1, axis=0), width*0.3, label = "Valid" )
    bar3 = ax.bar(x+width*0.3, np.count_nonzero(Y_test == 1, axis=0), width*0.3, label = "Test" )


    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Distribution')
    ax.set_title('Distribution overview of splitted dataset')
    ax.set_xticks(x)
    ax.set_xticklabels(mods)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
            fancybox=True, shadow=True, ncol=5)


    def autolabel(rects):
        """Attach a text label above each bar in *rects*, displaying its height."""
        for rect in rects:
            height = rect.get_height()
            ax.annotate('{}'.format(height),
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 0),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')

    # autolabel(bar1)
    # autolabel(bar2)
    # autolabel(bar3)
    # fig.tight_layout()
    return plt.show()

In [ ]:
def SNR_accuracy(in_snr, name):
    """
    Computes accuracies of chosen SNRs individualy

    in_snr:   must be list of SNRs
    """

    acc = []
    for snr in in_snr:
        acc_snr = snr
        idx_acc_snr = []

        for i in range(len(test_idx)):
            if int(lbl_SNR[test_idx[i]]) == int(acc_snr):
                idx_acc_snr.append(i)

        acc_X_test = X_test[idx_acc_snr]
        # acc_X_f_test = X_f_test[idx_acc_snr]
        acc_Y_test = Y_test[idx_acc_snr]

        print('\nSNR ' + str(acc_snr) + 'dB:')
        accuracy_snr = model.evaluate([acc_X_test], acc_Y_test, batch_size=32, verbose=2)
        acc.append(accuracy_snr)

    acc = np.vstack(acc)
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    plt.plot(SNR, (acc[:,1]*100), marker='.', markersize= 10, label = name, linestyle = '-',)
    ax.legend(loc=4, prop={'size': 15})

    x_major_ticks = np.arange(-20, 19, 2 )
    ax.set_xticks(x_major_ticks)

    y_major_ticks = np.arange(0, 101, 10 )
    y_minor_ticks = np.arange(0, 101, 2)
    ax.set_yticks(y_major_ticks)
    ax.set_yticks(y_minor_ticks, minor=True)
    ax.tick_params(axis='both', which='major', labelsize=10)

    ax.grid(which='both', linestyle='-')

    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)

    plt.xlim(-20, 18)
    plt.ylim(0,100)
    plt.title("Classification Accuracy",fontsize=10)
    plt.ylabel('Accuracy (%)',fontsize=10)
    plt.xlabel('SNR (dB)',fontsize=10)
#     plt.savefig(base_dir + name + '.png')
    plt.savefig("VGG.png",dpi=300)
#     files.download("VGG.png")
    plt.show()

    return acc[:,1]


In [ ]:
def layer_overview(model):

    """
    Offers overview of the model's layers and theirs outputs

    model: specify trained model you want to have overview of
    """

    # Names and outputs from layers
    layer_names = [layer.name for layer in model.layers]
    layer_outputs = [layer.output for layer in model.layers[:]]

    return layer_names, layer_outputs

def model_visualization(nth_layer, nth_test_idx, mods, model,
                        plot_sample = False, plot_activations = True,
                        plot_feature_maps = True):

    """
    The function provised overview of activation of specific layer and its
    feature maps.

    nth_layer: enter number which corresponds with the position of wanted layer
    nth_test_idx: enter number pointing at the test indexes from earlier
    mods: provide variable which holds listed modulations
    model: specify which trained model to load
    plot_sample = False: set to true to plot sample data
    plot_activations = True: plots activation of chosen layer
    plot_feature_maps = True: plots feature map of chosen layer
    """

    # Sample data for visualization
    test_sample = X_test[nth_test_idx,:,:] # shape [128,2]
    test_sample = test_sample[None] # change to needed [1,128,2]
    SNR = lbl_SNR[test_idx[nth_test_idx]]
    mod = one_hot[test_idx[nth_test_idx]]
    f, u = factorize(mods)
    mod = mod.dot(u)

    # Names and outputs from layers
    layer_names = [layer.name for layer in model.layers]
    layer_outputs = [layer.output for layer in model.layers[:]]

    ## Activations ##

    # define activation model
    activation_model = tf.keras.models.Model(model.input, layer_outputs)

    # get the activations of chosen test sample
    activations = activation_model.predict(test_sample)

    ## Feature-maps ##

    # define feature maps model
    feature_maps_model = tf.keras.models.Model(model.inputs, model.layers[4].output)

    # get the activated features
    feature_maps = feature_maps_model.predict(test_sample)


    # Plot sample
    if plot_sample == True:
        plt.plot(test_sample[0,:,:])
        plt.title(mod + '    ' + str(SNR) + 'dB')
        plt.show()

    # Plot activations
    if plot_activations == True:
        activation_layer = activations[nth_layer]
        activation_layer = np.transpose(activation_layer[:,:,:],(0,2,1)) # reshape
        fig, ax = plt.subplots(figsize=(20,10))
        ax.matshow(activation_layer[0,:,:],  cmap='viridis')
        # plt.matshow(activation_layer[0,:,:],  cmap='viridis')
        plt.title('Activation of layer ' + layer_names[nth_layer])
        ax.grid(False)
        ax.set_xlabel('Lenght of sequence')
        ax.set_ylabel('Filters')
        fig.show()


    # Plot feature maps
    if plot_feature_maps == True:
        n_filters = int(feature_maps.shape[2]/2); ix = 1
        fig = plt.figure(figsize=(25,15))
        for _ in range(n_filters):
            for _ in range(2):
                # specify subplot and turn of axis
                ax =fig.add_subplot(n_filters,  5, ix)
                # ax = plt.subplot(n_filters,  5, ix, )
                ax.set_xticks([])
                ax.set_yticks([])
                # plot filter channel in grayscale
                ax.plot(feature_maps[0, :, ix-1])
                ix += 1
        # show the figure
        fig.show()


In [ ]:
def merge_dict(dict1,dict2):
  return(dict2.update(dict1))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# with open('/content/drive/MyDrive/Colab Notebooks/RML2016.10b.dat', 'rb') as f:
with open('/content/drive/MyDrive/Colab Notebooks/RML2016.10b/RML2016.10b.dat', 'rb') as f:
    Xd = pickle.load(f, encoding='latin1')
sorted_data = {k: Xd[k] for k in sorted(Xd.keys())[0:200]}
data1={k: Xd[k] for k in sorted(sorted_data.keys())[0:20]}
data2={k: Xd[k] for k in sorted(sorted_data.keys())[40:60]}
merge_dict(data2,data1)
data={k: Xd[k] for k in sorted(sorted_data.keys())[100:180]}
merge_dict(data1,data)
#print(data)
print(list(data.keys()))

[('PAM4', -20), ('PAM4', -18), ('PAM4', -16), ('PAM4', -14), ('PAM4', -12), ('PAM4', -10), ('PAM4', -8), ('PAM4', -6), ('PAM4', -4), ('PAM4', -2), ('PAM4', 0), ('PAM4', 2), ('PAM4', 4), ('PAM4', 6), ('PAM4', 8), ('PAM4', 10), ('PAM4', 12), ('PAM4', 14), ('PAM4', 16), ('PAM4', 18), ('QAM16', -20), ('QAM16', -18), ('QAM16', -16), ('QAM16', -14), ('QAM16', -12), ('QAM16', -10), ('QAM16', -8), ('QAM16', -6), ('QAM16', -4), ('QAM16', -2), ('QAM16', 0), ('QAM16', 2), ('QAM16', 4), ('QAM16', 6), ('QAM16', 8), ('QAM16', 10), ('QAM16', 12), ('QAM16', 14), ('QAM16', 16), ('QAM16', 18), ('QAM64', -20), ('QAM64', -18), ('QAM64', -16), ('QAM64', -14), ('QAM64', -12), ('QAM64', -10), ('QAM64', -8), ('QAM64', -6), ('QAM64', -4), ('QAM64', -2), ('QAM64', 0), ('QAM64', 2), ('QAM64', 4), ('QAM64', 6), ('QAM64', 8), ('QAM64', 10), ('QAM64', 12), ('QAM64', 14), ('QAM64', 16), ('QAM64', 18), ('QPSK', -20), ('QPSK', -18), ('QPSK', -16), ('QPSK', -14), ('QPSK', -12), ('QPSK', -10), ('QPSK', -8), ('QPSK', -6)

In [ ]:
# 10a
SNR, X, modulations, one_hot, lbl_SNR = load_dataset(data)
# 10b
# SNR, X, modulations, one_hot, lbl_SNR = load_RMLb_dataset(RMLb_X, RMLb_lbl)

In [ ]:
# with open('/content/drive/MyDrive/Colab Notebooks/RML2016.10b/RML2016.10b.dat', 'rb') as f:
# # with open('/content/drive/MyDrive/Colab Notebooks/RML2016.10a/RML2016.10a_dict.pkl', 'rb') as f:
#     Xd = pickle.load(f, encoding='latin1')
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], data.keys())))), [1,0])
# unwanted = ['8PSK', 'WBFM', 'CPFSK', 'GFSK','AM-DSB']
# mods=set(mods).difference(set(unwanted))
# mods=list(mods)
print(mods)
X = []
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(data[(mod,snr)])
        for i in range(data[(mod,snr)].shape[0]):
            lbl.append((mod,snr))

X = np.vstack(X)

['8PSK', 'BPSK', 'PAM4', 'QAM16', 'QAM64', 'QPSK']


In [ ]:
# unwanted = ['8PSK', 'WBFM', 'CPFSK', 'GFSK','AM-DSB']
# mods=set(mods).difference(set(unwanted))
# list(mods)
# print(mods)
# SNR, X, modulations, one_hot, lbl_SNR = load_dataset(Xd)

In [ ]:
def to_onehot(yy):

    data = list(yy)
    yy1 = np.zeros([len(data), max(data)+1])
    yy1[np.arange(len(data)),data]=1

    return yy1

In [ ]:
# np.random.seed(2017)
# n_examples = X.shape[0]
# n_train = int(n_examples * 0.7)
# train_idx = np.random.choice(range(0,n_examples), size=n_train, replace=False)
# test_idx = list(set(range(0,n_examples))-set(train_idx))
# X_train = X[train_idx]
# X_test =  X[test_idx]
# Y_train = to_onehot(map(lambda x: mods.index(lbl[x][0]), train_idx))
# Y_test = to_onehot(map(lambda x: mods.index(lbl[x][0]), test_idx))
# in_shp = list(X_train.shape[1:])
# classes = mods

In [ ]:
train_idx, valid_idx, test_idx, X_train, X_valid, X_test, Y_train, Y_valid, Y_test = train_test_valid_split(X, one_hot, train_split=0.8, valid_split=0.1, test_split=0.1)

In [ ]:
X_train, X_valid, X_test = normalize_data(X_train, X_valid, X_test)

In [ ]:
# Set up some params
#nb_epoch =   45   # number of epochs to train on
#batch_size = 1024  # training batch size
#dr = 0.3 # dropout rate (%)#

In [ ]:
# in_shp = list(X_train.shape[1:])


In [ ]:
def resnet_model(in_shp):
    # Input layer
    # Reshape(in_shp+[1], input_shape=in_shp)
    inputs = tf.keras.Input(shape=in_shp)
    # Convolutional block 1
    x = layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu')(inputs,inputs)
    x = layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.BatchNormalization()(x)
    # Residual blocks
    num_blocks = 4  # Number of residual blocks
    num_filters = 64  # Number of filters in each residual block
    for _ in range(num_blocks):
        # Residual connection
        residual = x
        # Convolutional block
        x = layers.Conv2D(num_filters, kernel_size=(3, 3), padding='same', activation='relu')(x)
        x = layers.Conv2D(num_filters, kernel_size=(3, 3), padding='same', activation='relu')(x)
        x = layers.BatchNormalization()(x)
        # Skip connection
        x = layers.add([x, residual])
        x = layers.Activation('relu')(x)
    # Global average pooling
    x = layers.GlobalAveragePooling2D()(x)
    # Fully connected layers
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(10, activation='softmax')(x)
    # Create the model
    model = tf.keras.Model(inputs=inputs, outputs=x)
    return model

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "model.h5", save_best_only=True, monitor="val_loss"),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.3, patience=3, min_lr=0.00007),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=1)]

optimizer = keras.optimizers.Adam(learning_rate=0.0007)
# input_shape=(128,2,1)
in_shp = list(X_train.shape[:])
print(in_shp)
# num_classes=10
modelsa=resnet_model(in_shp)
modelsa.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

[576000, 2, 128]


TypeError: ignored

In [ ]:
modelsa.summary()

In [ ]:
tf.keras.backend.clear_session()
history = modelsa.fit(X_train, Y_train, batch_size=128, epochs=30, verbose=2, validation_data= (X_valid, Y_valid), callbacks=callbacks)

In [ ]:
# layer_overview(model)

In [ ]:
filepath = 'weight_4layers.wts.h5'
history = model.fit(X_train,Y_train,batch_size=batch_size,
    epochs=nb_epoch,
    verbose=1,
    validation_data=(X_test, Y_test),
    callbacks = [
        keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
        # keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
    ])

In [ ]:
model = keras.models.load_model("model.h5")

test_loss, test_acc = model.evaluate(X_test, Y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

In [ ]:
SNR_accuracy(SNR, 'ResNet')

In [ ]:
All_SNR_show_confusion_matrix([X_test], save=True)

In [ ]:
count=-20

In [ ]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues, labels=True, annot=False, font_size=10,count=0):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=font_size+2)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45, fontsize=font_size)
    plt.yticks(tick_marks, labels, fontsize=font_size)
    if annot:
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                plt.text(j, i, format(cm[i, j], '.2f'),
                         horizontalalignment="center",
                         color="white" if cm[i, j] > cm.max() / 2. else "black",
                         fontsize=font_size)

    plt.tight_layout()
    plt.ylabel('True label', fontsize=font_size)
    plt.xlabel('Predicted label', fontsize=font_size)
    path="VGG-SNR "+ str(count)+".png"
    plt.savefig(path,dpi=300)
    plt.show()

In [ ]:
acc = {mod: {} for mod in mods}
for snr in snrs:
  # extract classes @ SNR
  test_SNRs = map(lambda x: lbl[x][1], test_idx)
  test_SNRs = list(test_SNRs)
  test_X_i = X_test[np.where(np.array(test_SNRs)==snr)]
  test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]

  # estimate classes
  #tf.config.run_functions_eagerly(True)
  test_Y_i_hat = model.predict(test_X_i)
  conf = np.zeros([len(classes),len(classes)])
  confnorm = np.zeros([len(classes),len(classes)])

  for i in range(0,test_X_i.shape[0]):
    j = list(test_Y_i[i,:]).index(1)
    k = int(np.argmax(test_Y_i_hat[i,:]))
    conf[j,k] = conf[j,k] + 1

  for i in range(0,len(classes)):
    confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])

  acc_class = np.diag(confnorm)
  print(acc_class)
  plt.figure()
  plot_confusion_matrix(confnorm, labels=classes, title="Confusion Matrix (SNR=%d)"%(snr), annot=True,font_size=10, count=count)
  count+=2
  cor = np.sum(np.diag(conf))
  ncor = np.sum(conf) - cor
  print ("SNR: ",snr, " Overall Accuracy: ", cor / (cor + ncor))
  acc[snr] = 1.0 * cor / (cor + ncor)
#   for mod in mods:
#     acc[mod][snr] = 1.0 * cor / (cor + ncor)
#   plt.figure(figsize=(4, 4))
# # Loop through each modulation and plot
#   for mod in mods:
#       snrs_values = sorted(list(acc[mod].keys()))
#       accuracy_values = [acc[mod][snr] for snr in snrs_values]
#       plt.plot(snrs_values, accuracy_values, marker='o', label=mod)

#   plt.xlabel('SNR')
#   plt.ylabel('Accuracy')
#   plt.title('Accuracy for Different Modulations Across SNR values')
#   # plt.legend(loc='upper left')
#   plt.grid(True)
#   plt.show()


In [ ]:
# Dictionary to store accuracy for each modulation at each SNR
mod_acc = {}
for mod in mods:
    mod_acc[mod] = {}

for snr in snrs:
    test_SNRs = list(map(lambda x: lbl[x][1], test_idx))
    test_X_i = X_test[np.where(np.array(test_SNRs)==snr)]
    test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]

    # estimate classes
    test_Y_i_hat = model.predict(test_X_i)
    conf = np.zeros([len(classes),len(classes)])

    for i in range(0,test_X_i.shape[0]):
        j = list(test_Y_i[i,:]).index(1)
        k = int(np.argmax(test_Y_i_hat[i,:]))
        conf[j,k] = conf[j,k] + 1

    # Normalizing the confusion matrix
    confnorm = np.zeros([len(classes), len(classes)])
    for i in range(0,len(classes)):
        confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])

    # Saving accuracy for each modulation
    for idx, mod in enumerate(mods):
        mod_acc[mod][snr] = confnorm[idx, idx]

# Plotting
plt.figure(figsize=(10,8))
for mod in mods:
    snrs_values = sorted(list(mod_acc[mod].keys()))
    accuracy_values = [mod_acc[mod][snr] for snr in snrs_values]
    plt.plot(snrs_values, accuracy_values, marker='o', label=mod)

# Set titles, labels, and legends
plt.xlabel('SNR')
plt.ylabel('Accuracy')
plt.title('Modulation Specific Accuracy Across SNR values')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()
path="VGG-SNR mod vs  snr".png"
plt.savefig(path,dpi=300)